In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("ps6.ipynb")

In [2]:
import sqlite3

## Question 1: Basic SQL
In this problem, you'll interact with a toy SQL database using Python's
built-in `sqlite3` package. Documentation can be found at
<https://docs.python.org/3/library/sqlite3.html>. For this problem,
we'll use a popular toy SQLite database, called Chinook, which
represents a digital music collection. See the documentation at <https://github.com/lerocha/chinook-database/blob/master/README.md>
for a more detailed explanation. We'll use the `chinook.sqlite` file:

In [3]:
con = sqlite3.connect('chinook.sqlite')
cur = con.cursor()

**1(a)** (1 pt) Load the database using the Python `sqlite3` package. How many tables are in the database? Save the answer in the variable `n_tables`.

In [4]:
cur.execute("""SELECT name FROM sqlite_master WHERE type='table';""")
n_tables = len(cur.fetchall())

In [5]:
grader.check("q1a")

q1a results: All test cases passed!

**1(b)** (2 pts) What are the names of the tables in the database? Save the answer as
    a set of strings, `table_names`. **Note:** you should write Python `sqlite3`
    code to answer this; don't just look up the answer in the
    documentation!


In [6]:
cur.execute("""SELECT name FROM sqlite_master WHERE type='table';""")
table_names = set()
for table in cur.fetchall():
    table_names.add(table[0])

In [8]:
grader.check("q1b")

q1b results: All test cases passed!

**1(c)** (2 pts) Write a function `albums_starting_with(c)` that takes as an argument a single character `c` and
    returns a list of the primary keys (AlbumIds) of all the albums whose titles
    start with that character. Your function should ignore case, so that
    the inputs "a" and "A" yield the same results. Include error
    checking that raises an error in the event that the input is not a
    single character.


In [9]:
def albums_starting_with(c):
    if not isinstance(c, str):
        raise TypeError("input is not string")
    if len(c) != 1:
        raise ValueError("input is not a single character")
    cur.execute('''SELECT AlbumId FROM Album WHERE UPPER(Title) LIKE UPPER("''' + c + '''%")''')
    return [i[0] for i in cur.fetchall()]

In [10]:
grader.check("q1c")

q1c results: All test cases passed!

**1(d)** (2 pts) Write a function `songs_starting_with(c)` that takes as an argument a single character and
    returns a list of the primary keys (TrackIds) of all the **songs** whose album
    names begin with that letter. Again, your function should ignore
    case and perform error checking as in the previous exercise (again ignoring case).
    **Hint:** you'll need a JOIN statement here. Don't forget that you
    can use the `cursor.description` attribute to find out about tables
    and the names of their columns.


In [11]:
def songs_starting_with(c):
    if not isinstance(c, str):
        raise TypeError("input is not string")
    if len(c) != 1:
        raise ValueError("input is not a single character")
    cur.execute('''SELECT TrackId FROM Album JOIN Track ON Album.AlbumId = Track.AlbumId WHERE UPPER(Title) LIKE UPPER("''' + c + '''%")''')
    return [i[0] for i in cur.fetchall()]

In [12]:
grader.check("q1d")

q1d results: All test cases passed!

**1(e)** (2 pts) Write a function `cost_of(c)` that takes as an argument a single character and
    returns the cost of buying every song whose album begins with that
    letter. This cost should be based on the tracks' unit prices, so
    that the cost of buying a set of tracks is simply the sum of the
    unit prices of all the tracks in the set. Again your function should
    ignore case and perform appropriate error checking. 


In [13]:
def cost_of(c):
    if not isinstance(c, str):
        raise TypeError("input is not string")
    if len(c) != 1:
        raise ValueError("input is not a single character")
    cur.execute('''SELECT Inv.UnitPrice FROM (SELECT DISTINCT TrackId, UnitPrice FROM InvoiceLine) AS Inv
                LEFT JOIN Track ON Track.TrackId = Inv.TrackId
                LEFT JOIN Album ON Track.AlbumId = Album.AlbumId 
                WHERE UPPER(Title) LIKE UPPER("''' + c + '''%")''')
    res = cur.fetchall()
    return sum([i[0] for i in res])

In [14]:
grader.check("q1e")

q1e results: All test cases passed!

## Question 2: Additional SQL
In this case, we will use the included file `flights.sqlite`

In [15]:
con = sqlite3.connect('flights.sqlite')
cur = con.cursor()

Each question asks you to write a SQL query that performs a certain operation. For example, if the question asks, "how many flights are there", an answer could be 

> `SELECT COUNT(*) FROM flights`

Some questions refer to a parameter $x$. For these, you should write a query containing a placeholder, which we can run for different values of $x$. For example, if the question asks, "how many flights are there in month $x$?", where $x\in\{1,2,\dots,12\}$ is a numerical month, an answer could be 

> `SELECT COUNT(*) FROM flights WHERE month=?`.

For answers that ask for a single datum, we will run 
```
> cur.execute(your_query)
> result = cur.fetchone()
```
For answers that ask for a table or set of rows, we will use `cur.fetchall(...)` instead of `cur.fetchone(...)`. For answers that accept a parameter, we will use e.g. `cur.fetchone(..., [x])`.

**2(a)** (2 pts) What was the number of scheduled departures on day of the week $x$, where $x$ is the numerical day of the week (Sunday=0, Monday=1, etc.) Your query should return a tuple `(n, x)` where `n` is the number of flights, and `x` is the day of week. 
Hint: Use the time_hour column to get the week

In [16]:
avg_sched_dep = '''SELECT COUNT(*) as n, CAST(strftime('%w', time_hour, 'unixepoch') as integer) as week FROM flights GROUP BY week HAVING week=?'''

In [17]:
grader.check("q2a")

q2a results: All test cases passed!

**2(b)** (2 pts) Which month had the most scheduled departures? Your query should return tuple containing a single numerical month (for example `(2,)` if the answer is February.)

In [18]:
most_deps = '''SELECT m FROM (SELECT COUNT(*) as n, CAST(strftime('%m', time_hour, 'unixepoch') AS integer) AS m FROM flights GROUP BY m ORDER BY n DESC)'''

In [19]:
grader.check("q2b")

q2b results: All test cases passed!

**2(c)** (3 pts) On how many days was the highest recorded temperature below $x$ degrees?

In [20]:
high_above = '''SELECT COUNT(*) FROM (SELECT year || '-' || month || '-' || day AS date, MAX(temp) AS t FROM weather GROUP BY date) WHERE t<?'''

In [21]:
grader.check("q2c")

q2c results: All test cases passed!

**2(d)** (3 pts) In this dataset, a cancelled flight is one for which `dep_delay` is missing. What percentage of the flights were cancelled overall?

In [27]:
pct_cancelled = '''SELECT 1.0 - 1.0 * COUNT(dep_delay) / COUNT(1) FROM flights'''

In [28]:
grader.check("q2d")

q2d results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload this .zip file to Gradescope for grading.

In [26]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)